In [17]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import glob
from nltk.stem.porter import PorterStemmer

In [18]:
s = "Harga mangga per kg adalah 20.500."
s = re.sub(r'\w*n\w*','',s)
print(s)

Harga  per kg adalah 20.500.


In [19]:
def readFile(directory):
    listFile = glob.glob(directory+"/*.txt")
    return listFile

In [20]:
def readPerWord(file):
    words = []
    f = open(file, 'r') #open file
    for line in f:
        words.extend(line.split(' ')) #tiap satu baris split per kata
    
    f.close()

    return words

In [21]:
def removeEnter(listOfWords): #['<BODY>\n']
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].replace('\n','')
        
    return listOfWords

In [22]:
def removeNull(listOfWords): #['','','eat','food','','']
    listOfWords = list(filter(None, listOfWords))
    
    return listOfWords

In [23]:
def caseFolding(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].casefold()
        
    return listOfWords

<h3>Load Stop Word from NLTK</h3>

In [24]:
stop_words = stopwords.words('english')

In [25]:
def removeStopWord(listOfWords):
    for i in range(len(listOfWords)):
        if i in stop_words:
            listOfWords.remove(i)
        
    return listOfWords

<h3>Load Lemmatizer from NLTK</h3>

In [26]:
lemmatizer = WordNetLemmatizer()

In [27]:
def lemmatization(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = lemmatizer.lemmatize(listOfWords[i])
        
    return listOfWords

<h3>Load Stemming from NLTK</h3>

In [28]:
def stemming(listOfWords):
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in listOfWords]
    
    return stemmed

<h1>Test</h1>

In [29]:
listOfFiles = readFile('D:/Kuliah/Semester_6/STBI/New folder/searchEngineMiniProject/DataRouter')
listOfWordsOfFile = []

for i in listOfFiles: #iterasi tiap file
    listOfWords = readPerWord(i)
    listOfWords = removeEnter(listOfWords)
    listOfWords = removeNull(listOfWords)
    listOfWords = caseFolding(listOfWords)
    listOfWords = removeStopWord(listOfWords)
    listOfWords = lemmatization(listOfWords)
    listOfWords = stemming(listOfWords)

    listOfWordsOfFile.append(listOfWords)

# listOfWords = removeEnter(listOfWords)
    
for i in listOfWordsOfFile:
    print(i)
    print('--')

, 'consid', 'a', 'farm', 'credit', 'rescu', 'packag', 'by', 'easter', 'even', 'if', 'the', 'system', 'and', 'it', 'regul', 'do', 'not', 'ask', 'for', 'help', 'by', 'then.', '"we\'r', 'go', 'to', 'have', 'a', 'bill', 'to', 'markup,', 'i', 'guarante', 'you,', 'befor', 'the', 'easter', 'recess,"', 'sen.', 'david', 'boren', '(d-okla.)', 'said.', 'senat', 'major', 'leader', 'robert', 'byrd', '(d-w.va.)', 'want', 'recommend', 'on', 'farm', 'credit', 'present', 'by', 'april', '11,', 'when', 'congress', 'is', 'schedul', 'to', 'break', 'for', 'easter,', 'boren', 'said.', 'boren', 'urg', 'the', 'farm', 'credit', 'administr', '(fca),', 'the', "system'", 'regulator,', 'to', 'quickli', 'make', 'a', 'formal', 'request', 'for', 'aid.', 'under', 'the', '1985', 'farm', 'credit', 'law', 'pass', 'by', 'congress,', 'the', 'fca,', 'a', 'regulator,', 'is', 'to', 'certifi', 'when', 'the', 'system', 'ha', 'exhaust', 'all', 'it', 'capit', 'and', 'need', 'feder', 'govern', 'help.', 'however,', 'fca', 'chairman'